# Video HUD Processing Pipeline

This notebook demonstrates the complete pipeline for processing lab video analysis results into a HUD overlay video.

In [ ]:
import json
import sys
from pathlib import Path
import subprocess

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Import our processing modules
from video_processing.event_merger import merge_events_to_timeline
from video_processing.json_to_models import load_procedure_from_json, load_objective_events_from_json, load_analysis_events_from_json
from video_processing.video_hud_overlay import create_hud_video

## Step 1: Load and examine the original JSON files

In [ ]:
# Load the original analysis files
base_dir = Path.cwd().parent

# Load analysis events
with open(base_dir / "analysis_events_result.json", 'r') as f:
    analysis_data = json.load(f)
    
print(f"Analysis events: {len(analysis_data['events'])} events")
for i, event in enumerate(analysis_data['events'][:3]):  # Show first 3
    print(f"\nEvent {i+1}:")
    print(f"  Type: {'Warning' if 'warning_message' in event else 'Well State'}")
    print(f"  Time: {event['timestamp_range']}")
    if 'warning_message' in event:
        print(f"  Warning: {event['warning_message']}")
    elif 'well_id' in event:
        print(f"  Well: {event['well_id']} - {'Complete' if event['is_complete'] else 'Partial'}")

In [ ]:
# Load objective events
with open(base_dir / "objective_events_result.json", 'r') as f:
    objective_data = json.load(f)
    
print(f"Objective events: {len(objective_data['events'])} events")
for i, event in enumerate(objective_data['events'][:5]):  # Show first 5
    print(f"\nEvent {i+1}:")
    print(f"  Time: {event['timestamp_range']}")
    if 'new_setting_ul' in event:
        print(f"  Type: Pipette Setting - {event['new_setting_ul']}μL")
    elif 'reagent' in event:
        thinking = event['thinking'].lower()
        event_type = "Aspiration" if "aspirate" in thinking else "Dispensing"
        print(f"  Type: {event_type} - {event['reagent']['name']} ({event['reagent']['volume_ul']}μL)")
    else:
        print(f"  Type: Tip Change")

In [ ]:
# Load procedure
with open(base_dir / "procedure_result.json", 'r') as f:
    procedure_data = json.load(f)
    
print(f"Procedure goal wells: {len(procedure_data['goal_wells'])}")
for well in procedure_data['goal_wells']:
    reagents = [f"{r['name']} ({r['volume_ul']}μL)" for r in well['reagents']]
    print(f"  {well['well_id']}: {', '.join(reagents)}")
    
print(f"\nReagent sources: {', '.join(procedure_data['reagent_sources'])}")

## Step 2: Process events into timeline

In [ ]:
# Run the event merger
print("Merging events into timeline...")
merge_events_to_timeline(
    str(base_dir / "analysis_events_result.json"),
    str(base_dir / "objective_events_result.json"),
    str(base_dir / "procedure_result.json"),
    str(base_dir / "video_processing" / "merged_timeline.json")
)

In [ ]:
# Examine the merged timeline
with open(base_dir / "video_processing" / "merged_timeline.json", 'r') as f:
    timeline_data = json.load(f)

print(f"Total events in timeline: {timeline_data['total_events']}")
print("\nFirst 10 events:")
for i, event in enumerate(timeline_data['timeline'][:10]):
    print(f"  {i+1:2d}. {event['start_time']:.1f}s - {event['end_time']:.1f}s: {event['title']} [{event['event_type']}]")

## Step 3: Create HUD overlay video

In [ ]:
# Check if input video exists
input_video = base_dir / "videos" / "output_long_again_2_timestamped.mp4"
output_video = base_dir / "video_processing" / "hud_overlay_video.mp4"

print(f"Input video exists: {input_video.exists()}")
print(f"Input video path: {input_video}")
print(f"Output video path: {output_video}")

if input_video.exists():
    print("\nCreating HUD overlay video...")
    create_hud_video(
        str(input_video),
        str(base_dir / "video_processing" / "merged_timeline.json"),
        str(output_video)
    )
else:
    print("\nInput video not found. Please check the path.")

## Step 4: Verify output

In [ ]:
# Check if output video was created
if output_video.exists():
    print(f"✓ HUD overlay video created successfully!")
    print(f"File size: {output_video.stat().st_size / 1024 / 1024:.1f} MB")
    
    # Get video info
    try:
        cmd = ["ffprobe", "-v", "quiet", "-show_entries", "format=duration,size", "-of", "default=noprint_wrappers=1", str(output_video)]
        result = subprocess.run(cmd, capture_output=True, text=True)
        print("\nVideo info:")
        print(result.stdout)
    except:
        print("Could not get video info (ffprobe not available)")
else:
    print("❌ Output video not created")

## Event Statistics and Visualization

In [ ]:
# Analyze event distribution
import matplotlib.pyplot as plt
import numpy as np

# Count events by type
event_counts = {}
for event in timeline_data['timeline']:
    event_type = event['event_type']
    event_counts[event_type] = event_counts.get(event_type, 0) + 1

# Plot event distribution
plt.figure(figsize=(10, 6))
plt.bar(event_counts.keys(), event_counts.values())
plt.title('Event Distribution by Type')
plt.xlabel('Event Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Timeline visualization
plt.figure(figsize=(15, 8))
colors = {
    "warning": "red",
    "well_state": "lime",
    "pipette_setting": "yellow",
    "aspiration": "cyan",
    "dispensing": "orange", 
    "tip_change": "white"
}

y_pos = 0
for event in timeline_data['timeline']:
    start = event['start_time']
    duration = event['end_time'] - event['start_time']
    color = colors.get(event['event_type'], 'gray')
    
    plt.barh(y_pos, duration, left=start, color=color, alpha=0.7, 
             label=event['event_type'] if event['event_type'] not in [e.get_text() for e in plt.gca().get_legend_handles_labels()[1]])
    plt.text(start + duration/2, y_pos, event['title'][:20], 
             ha='center', va='center', fontsize=8, rotation=0)
    y_pos += 1

plt.xlabel('Time (seconds)')
plt.ylabel('Events')
plt.title('Event Timeline')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()